## Introduction



We&rsquo;re thinking about the problem of finding the cheapest possible
nutritious diet.  Last time we argued that this problem could be
expressed as a *linear program*
$$
    \min_x p'x
$$

such that
$$\begin{bmatrix}
      A\\
      -A
   \end{bmatrix}x \geq \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix},$$

where $p$ is a vector of prices, $A$ is a matrix that maps
vectors of quantities of food into vectors of nutrients, and where
$b_{min}$ and $b_{max}$ are respectively dietary minimums
and maximums of different nutrients.  As above, we will sometimes stack these
objects, obtaining
$$
      \tilde{A} = \begin{bmatrix}
                        A_{min}\\
                        -A_{max}
                      \end{bmatrix}
  $$
and
$$
      \tilde{b} = \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix}
  $$

Our job in this notebook: Specify the objects required by the linear
program $(p,\tilde{A},\tilde{b})$, then have the computer solve the problem for us.



## USDA Food Central DataBase



The USDA maintains a database of nutritional information, where
different kinds of food are identified by an FDC number.  They do
not provide any data on prices.  

To look up nutritional information, use api provided by the USDA at
[https://fdc.nal.usda.gov/](https://fdc.nal.usda.gov/).   You should sign up for a
free api key (see directions on page), then add that key here in
place of &ldquo;DEMO<sub>KEY</sub>&rdquo;.



In [170]:
apikey = "wBJH1Xcq7UxiJQWgw9WaP4nDb1FnJxzg6cebroAc"  # Replace with a real key!  "DEMO_KEY" will be slow...

### Looking up foods



I&rsquo;ve written a little module `fooddatacentral`.  Install it (only once!), along with other requirements.



In [171]:
%pip install -r requirements.txt --upgrade

  Using cached gspread-6.0.2-py3-none-any.whl (53 kB)
  Using cached StrEnum-0.4.15-py3-none-any.whl (8.9 kB)
Note: you may need to restart the kernel to use updated packages.


This module offers some simple methods

-   `search`
-   `nutrients`
-   `units`



### FDC Search



Here&rsquo;s a little code to help look up FDC codes for foods of
different descriptions.



In [172]:
import fooddatacentral as fdc

fdc.search(apikey,"frozen corn")

,fdcId,description,commonNames,additionalDescriptions,dataType,ndbNumber,publishedDate,foodCategory,allHighlightFields,score,...,foodNutrients,finalFoodInputFoods,foodMeasures,foodAttributes,foodAttributeTypes,foodVersionIds,foodCode,foodCategoryId,scientificName,mostRecentAcquisitionDate
0,173345,"Corn dogs, frozen, prepared",,,SR Legacy,22973.0,2019-04-01,"Meals, Entrees, and Side Dishes",,511.00372,...,"[{'nutrientId': 1057, 'nutrientName': 'Caffein...",[],[],[],[],[],NaN,NaN,NaN,NaN
1,2345442,"Corn, frozen, cooked with oil",,,Survey (FNDDS),NaN,2022-10-28,Corn,,511.00372,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Salt, table, iodized', '...",[{'disseminationText': 'Quantity not specified...,[],"[{'name': 'Adjustments', 'description': 'Adjus...",[],75216137.0,2650555.0,NaN,NaN
2,2345443,"Corn, frozen, cooked with butter or margarine",,animal fat;shortening,Survey (FNDDS),NaN,2022-10-28,Corn,,473.22314,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Salt, table, iodized', '...","[{'disseminationText': '1 baby corn', 'gramWei...",[],"[{'name': 'Attribute', 'description': 'Generic...",[],75216138.0,2650557.0,NaN,NaN
3,2345434,"Corn, frozen, cooked, no added fat",,cooking spray,Survey (FNDDS),NaN,2022-10-28,Corn,,473.22314,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Salt, table, iodized', '...",[{'disseminationText': 'Quantity not specified...,[],"[{'name': 'Additional Description', 'descripti...",[],75216112.0,2650539.0,NaN,NaN
4,168480,"Succotash, (corn and limas), frozen, unprepared",,,SR Legacy,11501.0,2019-04-01,Vegetables and Vegetable Products,,473.22314,...,"[{'nutrientId': 1062, 'nutrientName': 'Energy'...",[],[],[],[],[],NaN,NaN,NaN,NaN
5,169217,"Corn, yellow, whole kernel, frozen, microwaved",,,SR Legacy,11182.0,2019-04-01,Vegetables and Vegetable Products,,440.65400,...,"[{'nutrientId': 1211, 'nutrientName': 'Threoni...",[],[],[],[],[],NaN,NaN,NaN,NaN
6,169365,"Corn, sweet, white, frozen, kernels on cob, un...",,,SR Legacy,11913.0,2019-04-01,Vegetables and Vegetable Products,,412.28760,...,"[{'nutrientId': 1186, 'nutrientName': 'Folic a...",[],[],[],[],[],NaN,NaN,NaN,NaN
7,168400,"Corn, sweet, yellow, frozen, kernels on cob, u...",,,SR Legacy,11180.0,2019-04-01,Vegetables and Vegetable Products,,412.28760,...,"[{'nutrientId': 1186, 'nutrientName': 'Folic a...",[],[],[],[],[],NaN,NaN,NaN,NaN
8,2345438,"Corn, frozen, cooked, fat added, NS as to fat ...",,NS as to fat added,Survey (FNDDS),NaN,2022-10-28,Corn,,387.66727,...,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Oil or table fat, NFS', ...","[{'disseminationText': '1 regular ear', 'gramW...",[],"[{'name': 'Attribute', 'description': 'Generic...",[],75216122.0,2650547.0,NaN,NaN
9,170131,"Succotash, (corn and limas), frozen, cooked, b...",,,SR Legacy,11872.0,2019-04-01,Vegetables and Vegetable Products,,387.35970,...,"[{'nutrientId': 1079, 'nutrientName': 'Fiber, ...",[],[],[],[],[],NaN,NaN,NaN,NaN


### FDC Nutrients



Once we know the `fdc_id` of a particular food we can look up a
variety of information on it.  We start with nutrients



In [173]:
id = 2344957     # Put an FDC ID HERE!
fdc.nutrients(apikey,fdc_id=id)

,Quantity,Units
Protein,2.50,g
Total lipid (fat),14.10,g
"Carbohydrate, by difference",23.20,g
Energy,225.00,kcal
"Alcohol, ethyl",0.00,g
...,...,...
PUFA 20:5 n-3 (EPA),0.00,g
MUFA 22:1,0.00,g
PUFA 22:5 n-3 (DPA),0.00,g
"Fatty acids, total monounsaturated",5.37,g


### FDC Ingredients



We can also look up the ingredients for many foods in the FDC:



In [174]:
fdc.ingredients(apikey,id)

,Ingredient,Food Code/NDB Number,Weight (grams)
0,"Potatoes, french fried, all types, salt added ...",11403,100
1,"Vegetable oil, NFS",82101000,10


## Prices



Now, let&rsquo;s begin thinking about constructing the objects we need for
the linear program.  Start with specifying $p$, the vector of prices.  

I’ve looked up more recent prices for these same goods, and recorded these at https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/, in a sheet called “Stigler Table B (2022 Prices)”


Also note that some kinds of foods need to have unit weights (in
grams) supplied under &ldquo;Units&rdquo;; e.g., extra large eggs are taken to
each weigh 56g.  These conversions can also often be found on the USDA
FDC website.  

Food is purchased in particular units (gallons, pounds, grams).  And
in some cases the natural units are things like donuts or eggs, in
which case we may need to define our  own units (see the example of
&ldquo;xl<sub>egg</sub>&rdquo; below).  New units can be added to a file `.unitsrc` in your home directory.



### Example: Stigler&rsquo;s Foods



In [ ]:
%pip install gnupg

import pandas as pd
from eep153_tools.sheets import read_sheets

df = read_sheets("https://docs.google.com/spreadsheets/d/11MqhRmM_NcrqkT9OVsZxmeHbHcCcle3IJSjEBT8LA_M/edit?usp=sharing",sheet='Table A')

df = df.set_index('Food')

df

### Example: My Shopping Trip



Here&rsquo;s an example of describing some different kinds of food, along with
data on food prices.  This is all just based on a trip I took to the
grocery store, except that I&rsquo;ve used the USDA database to look up FDC
numbers.  Note that we may need extra information to map some units
into weights.  For example, I still need to weigh a crumpet.



#### Trip to Monterey Market



In [ ]:
import pandas as pd
from eep153_tools.sheets import read_sheets
df = read_sheets("https://docs.google.com/spreadsheets/d/11MqhRmM_NcrqkT9OVsZxmeHbHcCcle3IJSjEBT8LA_M/edit?usp=sharing",sheet='Table A')

df = df.set_index('Food')

df

### Units & Prices



Now, the prices we observe can be for lots of different quantities and
 units.  The FDC database basically wants everything in either hundreds
 of grams (hectograms) or hundreds of milliliters (deciliters).  

Sometimes this conversion is simple; if the price we observe is for
something that weighs two kilograms, that&rsquo;s just 20 hectograms.
Different systems of weights and volumes are also easy; a five pound
bag of flour is approximately 22.68 hectograms.  

Othertimes things are more complicated.  If you observe the price of a
dozen donuts, that needs to be converted to hectograms, for example.  

A function `units` in the [fdc](fooddatacentral.py) module accomplishes this conversion
for many different units, using the `python` [pint module](https://pint.readthedocs.io/en/latest/).  A file
[~/.units.rc](Data/food_units.txt) can be edited to deal with odd cases such as
donuts, using a format described in the `pint` [documentation](https://pint.readthedocs.io/en/latest/advanced/defining.html).

Here&rsquo;s an example of the usage of `fooddatacentral.units`:



In [ ]:
# Try your own quantities and units.
# If units are missing try adding to ~/.unitsrc

print(fdc.units(5,'lbs'))
print(fdc.units(1,'gallon'))
print(fdc.units(2,'tea_bag'))
print(fdc.units(12,'donut'))

Now, use the `units` function to convert all foods to either
 deciliters or hectograms, to match FDC database:



In [ ]:
import fooddatacentral as fdc

# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food')['FDC Price'].min()

Prices

## Mapping to Nutrients



Next we want to build the matrix $A$, which maps quantities of food
 into nutrients.  We have a list of foods with prices.  Do lookups on USDA database
 to get nutritional information.



In [ ]:
import warnings

D = {}
count = 0
for food in  df.index:
    try:
        FDC = df.loc[df.index==food,:].FDC.values[0]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
        print(food)
    except AttributeError:
        warnings.warn(f"Couldn't find FDC Code {FDC} for food {food}.")
D = pd.DataFrame(D,dtype=float)

D

## Dietary Requirements



We&rsquo;ve figured out some foods we can buy, the nutritional content of
those foods, and  the price of the foods.  Now we need to say
something about nutritional requirements, and construct the vectors
$b_{min}$ and $b_{max}$.   Our data for this is based
on  US government recommendations available at
[https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf](https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf)

I&rsquo;ve put some of these data into a google spreadsheet at
[https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/](https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/). 
Note that we&rsquo;ve tweaked the nutrient labels to match those in the FDC
data.

We&rsquo;ve broken down the requirements into three different tables.  The
first is *minimum* quantities that we need to  satisfy.  For example,
this table tells us that a 20 year-old female needs at least 46 grams
of protein per day.



In [ ]:
RDIs = read_sheets('https://docs.google.com/spreadsheets/d/1sOHRhvDZwvaPbLl8IBGY4faMwfP5i1CsQUJk3AbliUg/edit?usp=sharing')

bmin = RDIs['diet_minimums'].set_index('Nutrition')

# Drop string describing source
bmin = bmin.drop('Source',axis=1)

bmin

This next table specifies *maximum* quantities.  Our 20 year-old
female shouldn&rsquo;t have more than 2300 milligrams of sodium per day.



In [ ]:
bmax = RDIs['diet_maximums'].set_index('Nutrition')

# Drop string describing source
bmax = bmax.drop('Source',axis=1)

bmax

## Putting it together



Here we take the different pieces of the puzzle we&rsquo;ve developed and
put them together in the form of a linear program we can solve.
Recall that the mathematical problem we&rsquo;re trying to solve is
$$
    \min_x p'x
$$
such that
$$
     Ax \geq b
$$



### Objective function ($p$)



If we buy a bag of groceries with quantities given by $x$, the total
cost of the bag of groceries is the inner product of prices and
quantities.  Since we&rsquo;ve converted our units above, this gives us a
vector of prices where quantities are all in 100 g or ml units.



In [ ]:
p = Prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = p.index.intersection(D.columns)
p = p[use]

p

### Nutrient Mapping Matrix ($A$)



The matrix $A$ maps a bag of groceries $x$ into nutrients, but we
don&rsquo;t need to keep track of nutrients for which we don&rsquo;t have
contraints.



In [ ]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[p.index].fillna(0)
# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Maximum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])
A

### Constraint vector ($b$)



Finally, the right hand side vector $b$ in the expression
$$
    Ax\geq b
$$



In [ ]:

b = pd.concat([bmin,-bmax]) # Note sign change for max constraints

b

## Solving the problem



First, we find a solution to the problem



In [ ]:
from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

## Choose sex/age group!
group = "F 31-50"

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

result = lp(p, -A, -b[group], method='interior-point')

result

Let&rsquo;s interpret this.  Start with the cost of the solution:



In [ ]:
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")

Next, what is it we&rsquo;re actually eating?



In [ ]:
# Put back into nice series
diet = pd.Series(result.x,index=p.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

Given this diet, what are nutritional outcomes?



In [ ]:
tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b[group])})
print("\nWith the following nutritional outcomes of interest:")
tab

Finally, what are the constraints that bind?  Finding a less expensive
diet might involve finding less expensive sources for these particular nutrients.



In [ ]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())